In [20]:
import numpy as np
import os, sys, re
import matplotlib.pyplot as plt
import pandas as pd
import glob

from PyGEECSPlotter.scan_data_analysis import ScanDataAnalyzer
from PyGEECSPlotter.image_analysis import ImageAnalyzer


In [21]:
# Setting the diagnostic and analysis, display options for running 
# the analysis and for displaying data

diagnostic='CAM-HPD-CCD'
file_ext='.png'


analyzer_dict = {
    # 1) Saturation
    'return_n_saturated': False,  # Whether to count saturated pixels
    'bit_depth': 12,  # Bit depth of the image
    'saturation_threshold': None,  # Custom saturation threshold

    # 2) Background subtraction
    'bg_file': True,  # Whether to use a background file
    'bg_const': 0,  # Constant background subtraction value
}

display_dict = {}


In [22]:
def sort_and_extract_shotnumbers(file_list):
    # Extract shot number from "Image ###.png"
    def get_shot_num(path):
        filename = os.path.basename(path)
        match = re.search(r'Image (\d+)\.png', filename)
        return int(match.group(1)) if match else -1
    
    # Sort by shot number
    sorted_files = sorted(file_list, key=get_shot_num)
    
    # Extract shot numbers into a separate list
    shotnumbers = [get_shot_num(f) for f in sorted_files]
    
    return sorted_files, shotnumbers

In [31]:
# Load the Scalar Data for that scan in, and filter any data if desired

analyzer = ImageAnalyzer(diagnostic=diagnostic, 
                 file_ext=file_ext, 
                 analyzer_dict=analyzer_dict, 
                 display_dict=analyzer_dict
                        )



In [39]:
top_dir = r'N:\data\Y2025\08-Aug\25_0812'
kHz_ghost_dir = os.path.join(top_dir, '1643 ghost 100Hz png')

scan_data = pd.DataFrame()
file_list = sorted( glob.glob(os.path.join(kHz_ghost_dir, f'*{file_ext}')) )
scan_data['file_list'], scan_data['Shotnumber'] = sort_and_extract_shotnumbers(file_list)

x0_list, y0_list = np.zeros(len(file_list)),  np.zeros(len(file_list))
for i in range(len(scan_data)):
    filename = scan_data['file_list'][i]
    data = analyzer.load_data(filename)
    x0_list[i], y0_list[i] = ImageAnalyzer._get_centroid(data, thresh_min=0.1, thresh_max=4095)
    scan_data['x0'] = x0
    scan_data['y0'] = y0

scan_data.to_csv(os.path.join(kHz_ghost_dir, 'centroids.csv'), sep='\t')

In [40]:
scan_data

,file_list,Shotnumber,x0,y0
0,N:\data\Y2025\08-Aug\25_0812\1643 ghost 100Hz ...,0,152.421121,83.491385
1,N:\data\Y2025\08-Aug\25_0812\1643 ghost 100Hz ...,1,152.421121,83.491385
2,N:\data\Y2025\08-Aug\25_0812\1643 ghost 100Hz ...,2,152.421121,83.491385
3,N:\data\Y2025\08-Aug\25_0812\1643 ghost 100Hz ...,3,152.421121,83.491385
4,N:\data\Y2025\08-Aug\25_0812\1643 ghost 100Hz ...,4,152.421121,83.491385
...,...,...,...,...
3461,N:\data\Y2025\08-Aug\25_0812\1643 ghost 100Hz ...,3461,152.421121,83.491385
3462,N:\data\Y2025\08-Aug\25_0812\1643 ghost 100Hz ...,3462,152.421121,83.491385
3463,N:\data\Y2025\08-Aug\25_0812\1643 ghost 100Hz ...,3463,152.421121,83.491385
3464,N:\data\Y2025\08-Aug\25_0812\1643 ghost 100Hz ...,3464,152.421121,83.491385
